In [1]:
# import math
# import re
# import nltk
# import random
# import json
# import os
# from nltk.corpus import stopwords
# from nltk.tokenize import sent_tokenize
# from tqdm import tqdm

# from medvqa.datasets.qa_pairs_extractor import QuestionAnswerExtractor
# from medvqa.utils.common import CACHE_DIR

In [1]:
!python ../iuxray/extract_findings_and_impressions.py

Loading file /mnt/workspace/iu-x-ray/dataset/reports/reports.json ...
Extracing and merging findings and impressions ...
Saving to /home/pamessina/medvqa-workspace/cache/iuxray/findings+impression.txt ...
done!


In [2]:
!python ../mimiccxr/extract_findings_and_impressions.py

Loading MIMIC-CXR's report file paths ...
227835it [02:43, 1394.99it/s] 
Saving findings + impressions to /home/pamessina/medvqa-workspace/cache/mimiccxr/findings+impression.txt ...
100%|██████████████████████████████████| 227835/227835 [09:44<00:00, 390.11it/s]
Num failed paths = 419
Done!


In [501]:
from importlib import reload

In [590]:
from medvqa.utils.files import read_lines_from_txt
from medvqa.utils.common import CACHE_DIR
from medvqa.datasets.qa_pairs_extractor import QuestionAnswerExtractor
import medvqa
import re
import os
import random
import math
from tqdm import tqdm

In [586]:
reload(medvqa.datasets.qa_pairs_extractor)

<module 'medvqa.datasets.qa_pairs_extractor' from '/home/pamessina/medvqa/medvqa/datasets/qa_pairs_extractor.py'>

In [8]:
iuxray_raw_reports = read_lines_from_txt(os.path.join(CACHE_DIR, 'iuxray', 'findings+impression.txt'))

In [10]:
mimiccxr_raw_reports = read_lines_from_txt(os.path.join(CACHE_DIR, 'mimiccxr', 'findings+impression.txt'))

In [19]:
len(iuxray_raw_reports), len(mimiccxr_raw_reports)

(3927, 227416)

In [20]:
def print_qa_pairs(qa_pairs):
    for k, v in qa_pairs.items():
        print('------------')
        print('Q:', k)
        for s in v:
            print('A:',s)

In [695]:
qa_extractor = QuestionAnswerExtractor()

In [514]:
connectors = ['when', 'and', 'but', 'however', 'although', 'should', 'since', 'unless', 'nevertheless', 'although']
exp = '('+'|'.join(f'\\b{x}\\b' for x in connectors)+'|[,;.:])'
full_exp = f'{exp}+(\\s+{exp}+)*'
sep_regex = re.compile(full_exp, re.IGNORECASE)
sep_regex

re.compile(r'(\bwhen\b|\band\b|\bbut\b|\bhowever\b|\balthough\b|\bshould\b|\bsince\b|\bunless\b|\bnevertheless\b|\balthough\b|[,;.:])+(\s+(\bwhen\b|\band\b|\bbut\b|\bhowever\b|\balthough\b|\bshould\b|\bsince\b|\bunless\b|\bnevertheless\b|\balthough\b|[,;.:])+)*',
re.IGNORECASE|re.UNICODE)

In [515]:
def clean_sentence(self, sentence):
    last_sep = None
    i = 0
    n = len(sentence)
    clean_sentence = ''
    prepend_to_next_chunk = None
    for match in sep_regex.finditer(sentence):
        span = match.span()
        if i < span[0]:
            chunk = sentence[i:span[0]].strip()
            if prepend_to_next_chunk:
                if len(clean_sentence) > 0:
                    chunk = prepend_to_next_chunk + chunk
                prepend_to_next_chunk = None
            if self._is_relevant(chunk):
                if len(clean_sentence) > 0:
                    if last_sep:
                        clean_sentence += ' ' + last_sep
                    clean_sentence += ' '                    
                clean_sentence += chunk
            print('chunk =', chunk)
        i = span[1]
        last_sep = sentence[span[0]:span[1]]
        print('last_sep =', last_sep)
        if last_sep not in ('and', 'but', ':'):
            if last_sep == ', however,':
                last_sep = None
            else:
                done = False
                for x in ('should', 'since', 'unless', 'when'):
                    if x in last_sep:
                        prepend_to_next_chunk = last_sep + ' '
                        last_sep = None
                        done = True
                        break
                if not done:
                    last_sep = ','
    if i < n:
        chunk = sentence[i:].strip()
        if prepend_to_next_chunk:
            chunk = prepend_to_next_chunk + chunk
        if self._is_relevant(chunk):
            if len(clean_sentence) > 0:
                if last_sep:
                    clean_sentence += ' ' + last_sep
                clean_sentence += ' '
            clean_sentence += chunk
        print('chunk =', chunk)
    return clean_sentence

In [518]:
_s = 'AP chest compared to through : Pulmonary vascular congestion and mediastinal venous engorgement have progressed since .'
for x in sep_regex.finditer(_s):
    print(str(x))
print("$$$$"+clean_sentence(qa_extractor, _s)+"$$$$")
print("$$$$"+qa_extractor.clean_sentence(_s)+"$$$$")

<_sre.SRE_Match object; span=(29, 30), match=':'>
<_sre.SRE_Match object; span=(61, 64), match='and'>
<_sre.SRE_Match object; span=(112, 119), match='since .'>
chunk = AP chest compared to through
last_sep = :
chunk = Pulmonary vascular congestion
last_sep = and
chunk = mediastinal venous engorgement have progressed
last_sep = since .
$$$$Pulmonary vascular congestion and mediastinal venous engorgement have progressed$$$$
$$$$Pulmonary vascular congestion and mediastinal venous engorgement have progressed$$$$


In [23]:
tmp = qa_extractor.generate_qa_pairs('Lung volumes continues to be low.  The heart continues to be moderately enlarged with unchanged mild to moderate edema.  Moderate bilateral, right greater than left, pleural effusions are unchanged.  ET tube is in appropriate position, and the gastric tube ends in the stomach.  Left central venous line is in appropriate position.  Right PICC line ends at the lower SVC. Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions. ', debug=True)
print_qa_pairs(tmp)

------------
Q: what about the lungs?
A: Lung volumes continues to be low.
A: Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions.
------------
Q: what about the heart?
A: The heart continues to be moderately enlarged with unchanged mild to moderate edema.
------------
Q: what about edema?
A: The heart continues to be moderately enlarged with unchanged mild to moderate edema.
A: Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions.
------------
Q: what about pleural space?
A: Moderate bilateral, right greater than left, pleural effusions are unchanged.
A: Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions.
------------
Q: what about tubes and lines?
A: ET tube is in appropriate position, and the gastric tube ends in the stomach.
A: Left central venous line is in appropriate position.
A: Right PICC line ends at the lower SVC.
------------
Q: what about support devices and fo

In [507]:
tmp = qa_extractor.generate_qa_pairs('Lung volumes continues to be low.  The heart continues to be moderately enlarged with unchanged mild to moderate edema.  Moderate bilateral, right greater than left, pleural effusions are unchanged.  ET tube is in appropriate position, and the gastric tube ends in the stomach.  Left central venous line is in appropriate position.  Right PICC line ends at the lower SVC. Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions. ', debug=True)
print_qa_pairs(tmp)

------------
Q: what about the lungs?
A: Lung volumes continues to be low.
A: Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions.
------------
Q: what about the heart?
A: The heart continues to be moderately enlarged with unchanged mild to moderate edema.
------------
Q: what about edema?
A: The heart continues to be moderately enlarged with unchanged mild to moderate edema.
A: Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions.
------------
Q: what about pleural space?
A: Moderate bilateral, right greater than left, pleural effusions are unchanged.
A: Unchanged low lung volumes, mild pulmonary edema and bilateral moderate pleural effusions.
------------
Q: what about tubes and lines?
A: ET tube is in appropriate position, and the gastric tube ends in the stomach.
A: Left central venous line is in appropriate position.
A: Right PICC line ends at the lower SVC.
------------
Q: what about support devices and fo

In [697]:
report = random.choice(mimiccxr_raw_reports)
print(report)
print('=============================')
tmp = qa_extractor.generate_qa_pairs(report, debug=True)
print_qa_pairs(tmp)

Cardiomediastinal silhouette is within normal limits. The lungs are clear without focal consolidation, large effusion or pneumothorax. No congestion or edema. There is no fracture or dislocation. No free air below the right hemidiaphragm. No acute intrathoracic process.
** not captured: No acute intrathoracic process
------------
Q: what about the cardiac silhouette?
A: Cardiomediastinal silhouette is within normal limits
------------
Q: what about the mediastinum?
A: Cardiomediastinal silhouette is within normal limits
------------
Q: what about the lungs?
A: The lungs are clear without focal consolidation , large effusion or pneumothorax
------------
Q: what about pleural space?
A: The lungs are clear without focal consolidation , large effusion or pneumothorax
------------
Q: what about pneumothorax?
A: The lungs are clear without focal consolidation , large effusion or pneumothorax
------------
Q: what about consolidation?
A: The lungs are clear without focal consolidation , large 

In [637]:
_s = 'the enlarging pleural effusion has displaced the cardiomediastinal contours towards the left , limiting assessment of the heart size'
print("$$$$"+qa_extractor.clean_sentence(_s)+"$$$$")

$$$$the enlarging pleural effusion has displaced the cardiomediastinal contours towards the left$$$$


In [559]:
len(qa_extractor.questions), qa_extractor.questions

(91,
 ['what about COPD?',
  'what about abscess and cavitation?',
  'what about adenopathy?',
  'what about air collections?',
  'what about air space disease?',
  'what about air-fluid level?',
  'what about airways?',
  'what about ascites?',
  'what about aspiration?',
  'what about atelectasis?',
  'what about azygos lobe?',
  'what about azygos vein?',
  'what about bleeding?',
  'what about blurring?',
  'what about bones?',
  'what about bowel obstruction and loops?',
  'what about bronchiectasis?',
  'what about bronchogram?',
  'what about bronchovascular crowding?',
  'what about bullae and blebs?',
  'what about calcification?',
  'what about cancer or tumor?',
  'what about cardiomegaly?',
  'what about cholelithiasis?',
  'what about congestive heart failure CHF?',
  'what about consolidation?',
  'what about contrast?',
  'what about densities?',
  'what about edema?',
  'what about emphysema?',
  'what about fissures?',
  'what about fluid overload?',
  'what about flui

In [571]:
tmp = qa_extractor.search_by_unmatched(0, mimiccxr_raw_reports, 100)
x = tmp[1]
tmp

(2, 66019)

In [572]:
qa_extractor.generate_qa_pairs_compact_version(mimiccxr_raw_reports[x])

{'sentences': ['Portable upright AP radiograph of the chest',
  'There are low lung volumes',
  'Moderate right and small left bilateral pleural effusions persist , right pleural effusion appears decreased in extent',
  'There is also likely a component of bibasilar atelectasis',
  'Underlying infection in these locations cannot be ruled out',
  'The remainder of the lungs bilaterally is clear',
  'There is no pneumothorax',
  'A right PICC terminates in the mid SVC',
  'The cardiac size is unchanged',
  'Moderate right and small left pleural effusions with associated atelectasis',
  'Underlying infection in the lower lobes cannot be excluded'],
 'invalid': [],
 'unmatched': [0, 4],
 'matched': [1, 2, 3, 5, 6, 7, 8, 9, 10],
 'qa': {'80': [1, 3, 5, 9, 10],
  '52': [2, 9],
  '9': [3, 9],
  '57': [6],
  '89': [7],
  '67': [7],
  '90': [7],
  '78': [8]}}

In [696]:
# _report = ""
# print(_report)
# _qa_pairs = qa_generator.generate_question_answer_pairs(_report)
print(mimiccxr_raw_reports[x])
print("======================")
_qa_pairs = qa_extractor.generate_qa_pairs(mimiccxr_raw_reports[x], debug=True)
# _qa_pairs = qa_generator.remove_almost_duplicate_sentences(_qa_pairs)
print_qa_pairs(_qa_pairs)

Portable upright AP radiograph of the chest. There are low lung volumes. Moderate right and small left bilateral pleural effusions persist, although right pleural effusion appears decreased in extent.. There is also likely a component of bibasilar atelectasis. Underlying infection in these locations cannot be ruled out. The remainder of the lungs bilaterally is clear. There is no pneumothorax. A right PICC terminates in the mid SVC. The cardiac size is unchanged. Moderate right and small left pleural effusions with associated atelectasis. Underlying infection in the lower lobes cannot be excluded.
** not captured: Portable upright AP radiograph of the chest
------------
Q: what about the lungs?
A: There are low lung volumes
A: There is also likely a component of bibasilar atelectasis
A: The remainder of the lungs bilaterally is clear
A: Moderate right and small left pleural effusions with associated atelectasis
A: Underlying infection in the lower lobes cannot be excluded
------------


In [582]:
def get_informative_unmatched_sentences(corpus, qa_extractor, n_samples = 2000):
    
    indices = range(len(corpus))
    if len(corpus) > n_samples:
        indices = random.sample(indices, n_samples)
        
    unmatched_sentences = set()
    word2freq = dict()
    
    for i in tqdm(indices):
        for s in qa_extractor.get_unmatched_sentences(corpus[i]):
            unmatched_sentences.add(s)
            for w in re.split(r",?\s+", s):
                word2freq[w] = word2freq.get(w, 0) + 1
    
    max_freq = max(word2freq.values())    
    unmatched_sentences = list(unmatched_sentences)
    
    def score(s):
        score_max = 0
        score_sum = 0
        cnt = 0
        for w in re.split(r",?\s+", s):
            if w not in qa_extractor.stopwords:
                tmp = max_freq - word2freq.get(w, 0)
                score_max = max(tmp, score_max)
                score_sum += tmp
                cnt += 1
        return score_max * 0.4 + (score_sum / math.sqrt(max(cnt,1))) * 0.6
    
    unmatched_sentences.sort(key=score, reverse=True)
    return unmatched_sentences

In [672]:
unmatched_sentences = get_informative_unmatched_sentences(mimiccxr_raw_reports, qa_extractor, n_samples=1000)

100%|██████████| 1000/1000 [00:06<00:00, 147.73it/s]


In [673]:
len(unmatched_sentences)

190

In [681]:
qa_extractor.reload()

In [682]:
print_qa_pairs(qa_extractor.generate_qa_pairs('. '.join(unmatched_sentences[0:40]), debug=True))

---- invalid: The lateral view is difficult to evaluate given the arm-down positioning.
** not captured: A right-sided wide caliber sheath advanced via the internal jugular right approach remains in unchanged position
** not captured: Patient's clinical condition required examination in sitting semi-upright position using AP frontal and left lateral view
** not captured: clinically appropriate , standard PA and lateral radiographs are recommended with optimal inspiration in order to reassess
** not captured: Given the interval change in such a short time interval findings are likely consistent with mucous plugging causing obstruction
** not captured: In either event , lack of change in more than years indicates that this represents a benign process
** not captured: the involvement of more than one lobe is sometimes an indication of an atypical organism such as Legionella
** not captured: Status post non-ST elevation MI and exertional shortness of breath
** not captured: Alternatively t

In [683]:
!python ../../scripts/iuxray/generate_qa_pairs_from_reports.py

Loading original reports from /mnt/workspace/iu-x-ray/dataset/reports/reports.json
Processing each report ...
100%|██████████████████████████████████████| 3955/3955 [00:34<00:00, 114.79it/s]
Saving qa adapted reports to /home/pamessina/medvqa-workspace/cache/iuxray/qa_adapted_reports.json
Done!


In [684]:
!python ../../scripts/mimiccxr/generate_qa_pairs_from_reports.py

Processing MIMIC-CXR's original reports ...
227835it [50:06, 75.78it/s] 
Saving qa adapted reports to /home/pamessina/medvqa-workspace/cache/mimiccxr/qa_adapted_reports.json
Done!


## Generate summary file

In [685]:
from medvqa.datasets.iuxray import IUXRAY_QA_ADAPTED_REPORTS_JSON_PATH
from medvqa.datasets.mimiccxr import MIMICCXR_QA_ADAPTED_REPORTS_JSON_PATH
from medvqa.utils.files import load_json_file

In [686]:
iuxray_qa_adapted_reports = load_json_file(IUXRAY_QA_ADAPTED_REPORTS_JSON_PATH)

In [687]:
mimiccxr_qa_adapted_reports = load_json_file(MIMICCXR_QA_ADAPTED_REPORTS_JSON_PATH)

In [688]:
len(iuxray_qa_adapted_reports['questions']), len(iuxray_qa_adapted_reports['reports'])

(97, 3927)

In [689]:
len(mimiccxr_qa_adapted_reports['questions']), len(mimiccxr_qa_adapted_reports['reports'])

(97, 227835)

In [690]:
iuxray_qa_adapted_reports['reports'][1]

{'sentences': ['The cardiomediastinal silhouette is within normal limits for size and contour',
  'The lungs are normally inflated without evidence of focal airspace disease , pleural effusion , or pneumothorax',
  'Stable calcified granuloma within the right upper lung',
  'No acute bone abnormality , No acute cardiopulmonary process'],
 'invalid': [],
 'unmatched': [],
 'matched': [0, 1, 2, 3],
 'qa': {'79': [0],
  '87': [0],
  '86': [1, 2, 3],
  '56': [1],
  '61': [1],
  '5': [1],
  '21': [2],
  '38': [2],
  '84': [3],
  '15': [3]},
 'filename': '10.xml'}

In [694]:
def generate_summary_file(dataset, k, kk, output_path):
    
    q2a = [[] for _ in range(len(dataset['questions']))]
    n_invalid = 0
    unmatched_sentences = []
    for ri, report in enumerate(dataset['reports']):
        s = report['sentences']
        for q_idx, a_idxs in report['qa'].items():
            q_idx = int(q_idx)
            a_len = sum(len(s[i]) for i in a_idxs)
            q2a[q_idx].append((a_len, ri))
        n_invalid += len(report['invalid'])
        unmatched_sentences.extend(s[i] for i in report['unmatched'])
            
    sorted_questions = sorted([(len(a), i) for i,a in enumerate(q2a)], reverse=True)    
    
    with open(output_path, 'w') as file:
        file.write('Preguntas ordenadas por frecuencia (decreciente)\n\n')
        for f, q in sorted_questions:
            file.write('%d, %s\n' % (f, dataset['questions'][q]))
        file.write('\n')
        file.write('Ejemplos de respuestas por pregunta:')
        for i,(_,q) in enumerate(sorted_questions):
            answers = q2a[q]
            answers.sort()            
            tmp = []
            for _, a in answers:
                report = dataset['reports'][a]
                sentences = report['sentences']
                a_idxs = report['qa'][str(q)]
                answer = '. '.join(sentences[i] for i in a_idxs)
                tmp.append(answer)
            answers = tmp            
            n = len(answers)
            print("n=%d, %s" % (n, dataset['questions'][q]))
            if n > k:
                sample = []
                for j in range(k):
                    sample.append(answers[random.randint(int(n*j/k), int(n*(j+1)/k)-1)])
            else:
                sample = answers
            file.write('\n\n%d) %s:\n\n' % (i+1, dataset['questions'][q]))
            for a in sample:
                file.write('    A: %s\n' % a)
        file.write('\n\nTotal oraciones inválidas: %d\n' % n_invalid)
        file.write('Total oraciones sin match: %d\n' % len(unmatched_sentences))
        file.write('\n\nEjemplos de oraciones válidas que no hicieron match con ninguna pregunta\n\n')
        unmatched_sentences = list(set(unmatched_sentences))
        unmatched_sentences.sort(key=lambda s:len(s))
        nn = len(unmatched_sentences)
        for j in range(kk):
            file.write('    %s\n' % unmatched_sentences[random.randint(int(nn*j/kk), int(nn*(j+1)/kk)-1)])

In [692]:
generate_summary_file(iuxray_qa_adapted_reports, 10, 120, '../iuxray/iuxray_qa_dataset_summary_file.txt')

n=3514, what about the lungs?
n=3343, what about the heart?
n=3053, what about pleural space?
n=2686, what about pneumothorax?
n=2345, what about the mediastinum?
n=1934, what about bones?
n=1224, what about consolidation?
n=1180, what about the cardiac silhouette?
n=796, what about pulmonary vascularity?
n=714, what about opacities?
n=546, what about air space disease?
n=543, what about support devices and foreign bodies?
n=540, what about calcification?
n=435, what about granumola?
n=414, what about infiltrate?
n=389, what about thoracic aorta?
n=360, what about atelectasis?
n=348, what about edema?
n=345, what about tubes and lines?
n=328, what about nodules?
n=303, what about cardiomegaly?
n=257, what about pneumonia?
n=239, what about the diaphragm?
n=223, what about tortuosity?
n=220, what about surgery?
n=213, what about interstitial lung disease?
n=207, what about scarring?
n=207, what about pulmonary hila?
n=192, what about fractures?
n=190, what about the ribs?
n=182, what ab

In [693]:
generate_summary_file(mimiccxr_qa_adapted_reports, 10, 120, '../mimiccxr/mimiccxr_qa_dataset_summary_file.txt')

n=191641, what about the lungs?
n=184260, what about pleural space?
n=144619, what about pneumothorax?
n=129244, what about the heart?
n=117811, what about the mediastinum?
n=108717, what about support devices and foreign bodies?
n=97824, what about the cardiac silhouette?
n=88610, what about tubes and lines?
n=72867, what about bones?
n=69556, what about opacities?
n=69505, what about consolidation?
n=69001, what about atelectasis?
n=65524, what about edema?
n=54153, what about pulmonary hila?
n=53740, what about pulmonary vascularity?
n=52914, what about pneumonia?
n=31810, what about cardiomegaly?
n=28096, what about surgery?
n=25534, what about the diaphragm?
n=22222, what about thoracic aorta?
n=20327, what about vena cava?
n=17058, what about the stomach?
n=16204, what about the trachea?
n=15391, what about interstitial lung disease?
n=14336, what about calcification?
n=13946, what about the apical zone?
n=13616, what about fractures?
n=13425, what about infection?
n=11897, what 